In [9]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np

In [10]:
#=================
# Case 1) melt()
#=================
df_wide = pd.DataFrame({
    "region": ["East" , "West" , "East" , "West"] ,
    "month": ["2025-11" , "2025-11" , "2025-12" , "2025-12"] ,
    "sales": [120 , 90 , 150 , 110] ,
    "returns": [5 , 3 , 4 , 6] ,
})
df_wide

case1 = df_wide.melt(
    id_vars = ["region" , "month"] , value_vars = ["sales" , "returns"] ,
    var_name = "metric" , value_name = "value"
)
case1

,region,month,sales,returns
0,East,2025-11,120,5
1,West,2025-11,90,3
2,East,2025-12,150,4
3,West,2025-12,110,6


,region,month,metric,value
0,East,2025-11,sales,120
1,West,2025-11,sales,90
2,East,2025-12,sales,150
3,West,2025-12,sales,110
4,East,2025-11,returns,5
5,West,2025-11,returns,3
6,East,2025-12,returns,4
7,West,2025-12,returns,6


In [11]:
#==================================================
# Case 2) pivot() — long -> wide (unique mapping)
#==================================================
df_long_unique = pd.DataFrame({
    "region": ["East" , "East" , "West" , "West"] ,
    "month": ["2025-11" , "2025-12" , "2025-11" , "2025-12"] ,
    "product": ["Cable" , "Cable" , "Cable" , "Cable"] ,
    "sales": [120 , 150 , 90 , 110] ,
})
df_long_unique

case2 = df_long_unique.pivot(index = "region" , columns = "month" , values = "sales")
case2

,region,month,product,sales
0,East,2025-11,Cable,120
1,East,2025-12,Cable,150
2,West,2025-11,Cable,90
3,West,2025-12,Cable,110


month,2025-11,2025-12
region,,
East,120,150
West,90,110


In [12]:
#=========================================================
# Case 3) pivot_table() — long -> wide with aggregation
#=========================================================
df_txn = pd.DataFrame({
    "region": ["East" , "East" , "East" , "West" , "West" , "West"] ,
    "month": ["2025-11" , "2025-11" , "2025-12" , "2025-11" , "2025-11" , "2025-12"] ,
    "product": ["Cable" , "Cable" , "Cable" , "Wire" , "Wire" , "Wire"] ,
    "sales": [60 , 70 , 150 , 40 , 50 , 110] ,
})
df_txn

case3 = df_txn.pivot_table(
    index = "region" , columns = "month" , values = "sales" ,
    aggfunc = "sum" , fill_value = 0
)
case3

,region,month,product,sales
0,East,2025-11,Cable,60
1,East,2025-11,Cable,70
2,East,2025-12,Cable,150
3,West,2025-11,Wire,40
4,West,2025-11,Wire,50
5,West,2025-12,Wire,110


month,2025-11,2025-12
region,,
East,130,150
West,90,110


In [13]:
#================================================
# Case 4) stack/unstack — MultiIndex reshaping
#================================================
report = df_txn.pivot_table(
    index = ["region" , "product"] , columns = "month" ,
    values = "sales" , aggfunc = "sum" , fill_value = 0
)

case4_unstack = report.unstack("product")
case4_unstack

case4_stack = report.stack()
case4_stack

month   2025-11       2025-12       
product   Cable  Wire   Cable   Wire
region                              
East      130.0   NaN   150.0    NaN
West        NaN  90.0     NaN  110.0

region  product  month  
East    Cable    2025-11    130
                 2025-12    150
West    Wire     2025-11     90
                 2025-12    110
dtype: int64

In [14]:
# ============================================================
# Case 5) wide_to_long() — patterned columns -> tidy long
# ============================================================
df_pattern = pd.DataFrame({
    "id": [1 , 2 , 3] ,
    "sales_2025Q1": [100 , 120 , 90] ,
    "sales_2025Q2": [110 , 115 , 95] ,
    "returns_2025Q1": [5 , 6 , 4] ,
    "returns_2025Q2": [4 , 7 , 3] ,
})
df_pattern

case5 = pd.wide_to_long(
    df_pattern , stubnames = ["sales" , "returns"] , i = "id" ,
    j = "quarter" , sep = "_" , suffix = ".+"
).reset_index()
case5

,id,sales_2025Q1,sales_2025Q2,returns_2025Q1,returns_2025Q2
0,1,100,110,5,4
1,2,120,115,6,7
2,3,90,95,4,3


,id,quarter,sales,returns
0,1,2025Q1,100,5
1,2,2025Q1,120,6
2,3,2025Q1,90,4
3,1,2025Q2,110,4
4,2,2025Q2,115,7
5,3,2025Q2,95,3


In [15]:
#================================================
# Case 6) explode() — list-like column -> rows
#================================================
df_tags = pd.DataFrame({
    "post_id": [101 , 102 , 103] ,
    "title": ["Pandas basics" , "Joins in pandas" , "Time series tricks"] ,
    "tags": [["pandas" , "python"] , ["pandas" , "merge" , "etl"] , ["timeseries" , "pandas"]] ,
})
df_tags

case6 = df_tags.explode("tags" , ignore_index = True)
case6

,post_id,title,tags
0,101,Pandas basics,"[pandas, python]"
1,102,Joins in pandas,"[pandas, merge, etl]"
2,103,Time series tricks,"[timeseries, pandas]"


,post_id,title,tags
0,101,Pandas basics,pandas
1,101,Pandas basics,python
2,102,Joins in pandas,pandas
3,102,Joins in pandas,merge
4,102,Joins in pandas,etl
5,103,Time series tricks,timeseries
6,103,Time series tricks,pandas
